In [1]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense,Dropout,LSTM,Embedding,SimpleRNN, GRU,Activation,Flatten
from tensorflow.keras.layers import Conv1D,MaxPool1D, Flatten
from tensorflow.keras.utils import plot_model
from tensorflow.keras.preprocessing import sequence
from sklearn.metrics import (precision_score, recall_score,confusion_matrix,f1_score, accuracy_score,mean_squared_error,mean_absolute_error)
import pandas as pd
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, CSVLogger
import matplotlib.pyplot as plt

In [3]:
import tensorflow.keras
print('keras: %s' % tensorflow.keras.__version__)

keras: 2.6.0


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
df = pd.read_csv(r'C:\Users\Admin\Downloads\clean_data.csv')

In [6]:
df.head()

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Label_DDoS,Label_DoS GoldenEye,Label_DoS Hulk,Label_DoS Slowhttptest,Label_DoS slowloris,Label_FTP-Patator,Label_Heartbleed,Label_Infiltration,Label_PortScan,Label_SSH-Patator
0,0,-0.646799,0.324411,-1.948333,0.204423,2.428566,-0.662668,2.051947,-0.472445,-0.008076,...,0,0,0,0,0,0,0,0,0,0
1,1,1.783219,0.430907,0.508829,0.114303,-0.421518,1.505849,-0.394452,-0.469837,-0.009562,...,0,0,0,0,0,0,0,0,0,0
2,2,-0.302869,-0.414284,0.159378,-0.558287,-0.421518,1.505849,-0.416902,-0.472439,-0.009562,...,0,0,0,0,0,0,0,0,0,0
3,3,-0.126277,-1.034865,0.769223,-1.040696,-0.421518,1.505849,-0.407956,-0.470938,-0.006590,...,0,0,0,0,0,0,0,0,0,0
4,4,-0.708056,0.979845,-1.948333,-0.560275,2.462290,-0.662668,-0.401372,-0.472445,-0.011049,...,0,0,0,0,0,0,0,0,0,0


In [7]:
df = df.sample(frac = 1)

In [8]:
df.drop(df.columns[0] , inplace=True , axis=1)

In [9]:
df.columns

Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet Length', 'Max Packet Length', 'Packet Length Mean',
  

In [10]:
LabelColumns = ['Label_BENIGN', 'Label_Bot','Label_DDoS', 'Label_DoS GoldenEye', 'Label_DoS Hulk','Label_DoS Slowhttptest', 'Label_DoS slowloris', 'Label_FTP-Patator','Label_Heartbleed', 'Label_Infiltration', 'Label_PortScan','Label_SSH-Patator']

In [11]:
y = df[LabelColumns]

In [12]:
y

,Label_BENIGN,Label_Bot,Label_DDoS,Label_DoS GoldenEye,Label_DoS Hulk,Label_DoS Slowhttptest,Label_DoS slowloris,Label_FTP-Patator,Label_Heartbleed,Label_Infiltration,Label_PortScan,Label_SSH-Patator
534283,1,0,0,0,0,0,0,0,0,0,0,0
348536,1,0,0,0,0,0,0,0,0,0,0,0
409088,1,0,0,0,0,0,0,0,0,0,0,0
1201071,1,0,0,0,0,0,0,0,0,0,0,0
539033,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1005003,1,0,0,0,0,0,0,0,0,0,0,0
1283854,1,0,0,0,0,0,0,0,0,0,0,0
484687,1,0,0,0,0,0,0,0,0,0,0,0
1329171,0,0,1,0,0,0,0,0,0,0,0,0


In [13]:
X = df.drop(LabelColumns , axis=1)

In [14]:
X

,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,Total Backward Packets,...,act_data_pkt_fwd,min_seg_size_forward,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min
534283,2.274937,0.484155,0.422595,4.281044,-0.399227,-0.662668,1.061774,-0.166888,0.002327,0.000966,...,0.007494,0.002719,1.077384,-0.097503,0.635839,1.246965,-0.029419,-0.121018,-0.04588,-0.011585
348536,1.486786,0.428971,-0.947027,0.114303,-0.421518,1.505849,-0.420615,-0.471090,-0.009562,-0.007983,...,-0.006451,0.002735,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.40072,-0.380320
409088,-0.492718,0.424130,0.736433,-0.174612,-0.419975,-0.662668,-0.400022,-0.322957,-0.008076,-0.009102,...,-0.008194,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.40072,-0.380320
1201071,-0.125239,0.365073,0.116426,1.633759,-0.399227,-0.662668,-0.413864,-0.284767,-0.003618,-0.004627,...,-0.004708,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.40072,-0.380320
539033,-0.152393,-1.129743,-1.948333,0.079182,2.467663,-0.662668,-0.355966,-0.472445,-0.009562,-0.009102,...,-0.008194,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.40072,-0.380320
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1005003,-0.703718,0.372818,-1.965411,-0.357504,1.960211,-0.662668,-0.399009,-0.472313,-0.009562,-0.010221,...,-0.008194,0.002729,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.40072,-0.380320
1283854,1.823736,0.487059,0.678755,0.208399,-0.421518,1.505849,0.246308,-0.472443,-0.009562,-0.007983,...,-0.006451,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.40072,-0.380320
484687,0.269181,0.482219,0.656033,3.949719,-0.399227,-0.662668,3.762890,-0.470999,-0.006590,-0.009102,...,-0.002965,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.40072,-0.380320
1329171,-1.054933,-1.296264,-0.933007,-1.039370,-0.419975,-0.662668,-0.416227,-0.460942,-0.008076,-0.004627,...,-0.004708,0.002719,-0.128601,-0.097503,-0.146881,-0.106173,-0.394660,-0.121018,-0.40072,-0.380320


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

#  Convolutional Neural Networks

In [16]:
X_train.shape , y_train.shape , X_test.shape

((1240540, 83), (1240540, 12), (611013, 83))

In [17]:
type(X_train)

pandas.core.frame.DataFrame

In [18]:
X_train = X_train.to_numpy().reshape(1240540, 83,1)
X_test = X_test.to_numpy().reshape(611013, 83,1)


In [19]:
model_cnn = Sequential()
model_cnn.add(Conv1D(64, 3, padding="same",activation="relu",input_shape=(83, 1)))
model_cnn.add(Conv1D(64, 3, padding="same", activation="relu"))
model_cnn.add(MaxPool1D(pool_size=(2)))
model_cnn.add(Conv1D(128, 3, padding="same", activation="relu"))
model_cnn.add(Conv1D(128, 3, padding="same", activation="relu"))
model_cnn.add(MaxPool1D(pool_size=(2)))
model_cnn.add(Flatten())
model_cnn.add(Dense(128, activation="relu"))
model_cnn.add(Dropout(0.5))
model_cnn.add(Dense(12, activation="sigmoid"))

In [20]:
model_cnn.compile(loss="binary_crossentropy", optimizer="adam",metrics=['accuracy'])
es = EarlyStopping(monitor='val_accuracy', mode='max', min_delta=0.0001,patience=5) ## early stoppoing

#FIT AND SAVE MODEL FOR FUTURE
model_cnn.fit(X_train, y_train, epochs=1000,validation_data=(X_test, y_test),callbacks=[es])
model_cnn.save("model_cnn_model.hdf5")

Epoch 1/1000
38767/38767 [==============================] - 256s 7ms/step - loss: 0.0023 - accuracy: 0.9970 - val_loss: 4.2398e-04 - val_accuracy: 0.9995
Epoch 2/1000
38767/38767 [==============================] - 252s 6ms/step - loss: 0.0011 - accuracy: 0.9992 - val_loss: 2.8960e-04 - val_accuracy: 0.9998
Epoch 3/1000
38767/38767 [==============================] - 249s 6ms/step - loss: 0.0014 - accuracy: 0.9993 - val_loss: 0.0029 - val_accuracy: 0.9997
Epoch 4/1000
38767/38767 [==============================] - 256s 7ms/step - loss: 0.0030 - accuracy: 0.9993 - val_loss: 6.2305e-04 - val_accuracy: 0.9997
Epoch 5/1000
38767/38767 [==============================] - 252s 6ms/step - loss: 0.0059 - accuracy: 0.9993 - val_loss: 3.4982e-04 - val_accuracy: 0.9997
Epoch 6/1000
38767/38767 [==============================] - 269s 7ms/step - loss: 0.0020 - accuracy: 0.9990 - val_loss: 5.7575e-04 - val_accuracy: 0.9993
Epoch 7/1000
38767/38767 [==============================] - 363s 9ms/step - loss

# Accuracies Measure && Model Power

In [23]:
y_pred_cnn = model_cnn.predict(X_test)
y_pred_cnn

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       ...,
       [0.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00],
       [1.4873673e-21, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        0.0000000e+00, 0.0000000e+00]], dtype=float32)

In [24]:
def predict_prob(number):
  return [number[0],1-number[0]]
y_probs_cnn = np.array(list(map(predict_prob,y_pred_cnn )))
y_probs_cnn

array([[1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 1.00000000e+00],
       [1.00000000e+00, 0.00000000e+00],
       [1.48736727e-21, 1.00000000e+00]])

In [26]:
np.savetxt('cnn_predictions.txt', np.transpose(np.concatenate((y_test.to_numpy().reshape((y_test.size, 1)),y_pred_cnn), axis=1)), fmt='%01d')
np.savetxt('cnn_prob_predictions.txt', np.around(np.transpose(y_probs_cnn),decimals=5), fmt='%.5f')


ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 0, the array at index 0 has size 7332156 and the array at index 1 has size 611013

In [27]:
accuracy = accuracy_score(y_test, y_pred_cnn)
print("accuracy:",accuracy)
f1score=f1_score(y_test, y_pred_cnn)
print("f1-acore:",f1score)
cm=confusion_matrix(y_test, y_pred_cnn)
print("confusion matrix:\n",cm)
pr=precision_score(y_test,y_pred_cnn)
print("Precision:",pr)
rs=recall_score(y_test,y_pred_cnn)
print("Recall_score:",rs)

ValueError: Classification metrics can't handle a mix of multilabel-indicator and continuous-multioutput targets

In [28]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
cm = confusion_matrix(y_test, y_pred_cnn, labels=model_cnn.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=model_cnn.classes_)
plt.title("CNN-Confusion Matrix")
plt.show()

AttributeError: 'Sequential' object has no attribute 'classes_'

## ROC Curves

In [29]:
from sklearn.metrics import roc_curve, auc,roc_auc_score
plt.figure()
fpr, tpr, thresholds =  roc_curve(y_test, y_probs_cnn)
auc =  roc_auc_score(y_test,y_pred_cnn)
plt.plot(fpr, tpr, label='%s ROC (area = %0.2f)' % ('CNN', auc))
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('1-Specificity(False Positive Rate)')
plt.ylabel('Sensitivity(True Positive Rate)')
plt.title('ROC -- Deep Learning Algorithms')
plt.legend(loc="lower right")
plt.rcParams['figure.figsize']=(16,8)
plt.show()   # Display


ValueError: multilabel-indicator format is not supported

<Figure size 432x288 with 0 Axes>